# DICTIONARY START

In [169]:
diq = pd.read_excel('программы_партнерка.xlsx')

In [170]:
diq.columns = ['unused', 'IK', 'TM']
diq = diq.drop('unused', axis=1)

In [182]:
diq[~diq.IK.isna()].drop_duplicates('IK')

,IK,TM
3,нет,нет
4,ТМ факт_Альфа (если есть АПП),ТМ_Альфа
7,ТМ факт Альфа,ТМ_Альфа
15,Телемедицина_Ренессанс МКДР,Телемедицина в ДМС Ренессанс
19,РГС Телемедицина ЮЛ,РГС Телемедицина ЮЛ
...,...,...
322,ТМ_Премьер Партнер 2 года_МКДР,Премьер - Партнер 2 года
325,Коробочный продукт Взрослая телемедицина_АК-1 ...,ТМ Взрослая_АК_Коробочный продукт
335,ТМ_Индивид доступ200_деж+узк-2_6 мес_МКДР,Телемедицина_Полная программа
336,ТМ_Индивид доступ285_деж+узк_6 мес_МКДР,Телемедицина Расширенная программа


In [184]:
# pd.Series(diq[~diq.IK.isna()].drop_duplicates('IK').IK,index=diq[~diq.IK.isna()].drop_duplicates('IK').TM).to_dict()

In [186]:
dict_names = dict(zip(diq[~diq.IK.isna()].drop_duplicates('IK').IK, diq[~diq.IK.isna()].drop_duplicates('IK').TM))

In [187]:
dict_names

{'нет': 'нет',
 'ТМ факт_Альфа (если есть АПП)': 'ТМ_Альфа',
 'ТМ факт Альфа': 'ТМ_Альфа',
 'Телемедицина_Ренессанс МКДР': 'Телемедицина в ДМС Ренессанс',
 'РГС Телемедицина ЮЛ': 'РГС Телемедицина ЮЛ',
 'РГС Телемедицина ФЛ': 'РГС Телемедицина ФЛ',
 'ТМ_Согаз': 'ТМ_Согаз',
 'ИГС_базовая_ТМ': 'Телемед Базовая',
 'Премиум_Согласие_МКДР': 'Премиум_Согласие_МКДР',
 'Семейная_Согласие_МКДР': 'Семейная_Согласие_МКДР',
 'Телемедицина_Согласие_дс12': 'ТМ_Согласие_факт',
 'ТМ психолог': 'ТМ_Согласие_факт + Психология',
 'Базовая_Согласие_МКДР': 'Базовая_Согласие_МКДР',
 'Стандарт_Согласие_МКДР': 'Стандарт_Согласие_МКДР',
 'Индивидуальный доступ_МКДР': 'Индивидуальный доступ',
 'Телемед ЮЛ_МКДР': 'Телемедицина_ЮЛ',
 'РГС Телемедицина ФЛ + псих_Навигация по ОМС': 'РГС Телемедицина ФЛ   + Опция «Навигация по ОМС»',
 'РГС Телемедицина ФЛ - семейная': 'РГС Телемедицина ФЛ - семейная',
 'РГС Телемедицина ФЛ + Псих помощь': 'РГС Телемедицина ФЛ   + Психологическая помощь',
 'Второе медицинское мнение 

# DICTIONARY END

In [1]:
period_start = '01.01.2020'
period_end = '31.03.2020'

In [2]:
import pandas as pd

In [3]:
import fdb

In [4]:
import sys
sys.path.append("/home/sergey/drclinics/common")

In [5]:
sys.path.append("/home/sergey/drclinics/reports")

In [6]:
from universal_connection import UniversalConnection, DBType

In [7]:
from auto_width7 import auto_columns_width

In [8]:
from utils import get_path, DATETIME_FORMAT
import datetime

# DISCARDS

In [9]:
sql_discards = f"""

SELECT	clhistnum.PCODE, -- ПАТИЕНТ_ИД
		clhistnum.BDATE, -- НАЧАЛО ПРИКРЕПЛЕНИЯ
		clhistnum.FDATE, -- КОНЕЦ ПРИКРЕПЛЕНИЯ
		clhistnum.DATECANCEL, -- ДОСРОЧНОЕ ОТКРЕПЛЕНИЕ
		jpersons.JNAME, -- ПАРТНЕР
		jplists.shortname, -- ДОГОВОР
		JPAGRTYPES.TNAME, -- ТИП ДОГОВОРА
		clients.FULLNAME,
        clients.BDATE birthday
		--
FROM clhistnum
--LEFT JOIN TREAT ON treat.pcode = clhistnum.PCODE
--
left JOIN jplists ON clhistnum.LSTID = JPLISTS.LID
LEFT JOIN JPAGREEMENT ON JPAGREEMENT.AGRID = CLHISTNUM.AGRID
LEFT JOIN JPAGRTYPES ON JPAGRTYPES.AGRTYPE=JPAGREEMENT.AGRTYPE
LEFT JOIN JPERSONS on JPAGREEMENT.jid = jpersons.JID
--
LEFT JOIN clients ON clients.PCODE = clhistnum.PCODE
--
WHERE 
(clhistnum.FDATE >= '{period_start}' AND clhistnum.FDATE <= '{period_end}') 
OR (clhistnum.DATECANCEL >= '{period_start}' AND clhistnum.DATECANCEL <= '{period_end}' AND clhistnum.FDATE > '{period_end}')


"""

In [10]:
conn = UniversalConnection('../../../../.credentials/infoclinic/PROM.cfg', DBType.FireBird)
df_discards = conn.query(sql_discards)
conn.close()

2020-06-19 13:38:01 connect to firebird database using config file "../../../../.credentials/infoclinic/PROM.cfg"
2020-06-19 13:38:02 connected to database:
                    b:\ifk-fdb\maindb\099.fdb
2020-06-19 13:38:02 query firebird:
                    
				    
				    SELECT	clhistnum.PCODE, -- ПАТИЕНТ_ИД
				    		clhistnum.BDATE, -- НАЧАЛО ПРИКРЕПЛЕНИЯ
				    		clhistnum.FDATE, -- КОНЕЦ ПРИКРЕПЛЕНИЯ
				    		clhistnum.DATECANCEL, -- ДОСРОЧНОЕ ОТКРЕПЛЕНИЕ
				    		jpersons.JNAME, -- ПАРТНЕР
				    		jplists.shortname, -- ДОГОВОР
				    		JPAGRTYPES.TNAME, -- ТИП ДОГОВОРА
				    		clients.FULLNAME,
				            clients.BDATE birthday
				    		--
				    FROM clhistnum
				    --LEFT JOIN TREAT ON treat.pcode = clhistnum.PCODE
				    --
				    left JOIN jplists ON clhistnum.LSTID = JPLISTS.LID
				    LEFT JOIN JPAGREEMENT ON JPAGREEMENT.AGRID = CLHISTNUM.AGRID
				    LEFT JOIN JPAGRTYPES ON JPAGRTYPES.AGRTYPE=JPAGREEMENT.AGRTYPE
				    LEFT JOIN JPERSONS on JPAGRE

In [11]:
df_discards

,PCODE,BDATE,FDATE,DATECANCEL,JNAME,SHORTNAME,TNAME,FULLNAME,BIRTHDAY
0,990159928,2019-02-12,2020-02-11,NaT,ФЛ,"ППП:Программа медобслуживания для взрослых ""Ст...",ФЛ предоплата,Балахнина Елена Анатольевна,1959-04-07
1,995133290,2019-02-01,2020-01-31,NaT,Закрытое акционерное общество «МетЛайф»,АПП Метлайф,СК 1 факт,Двинских Александр Эдуардович,1993-09-05
2,990129261,2019-01-15,2020-01-16,NaT,ФЛ,"ППП:Программа медобслуживания для взрослых ""Ст...",ФЛ предоплата,Замятина Ольга Игоревна,1987-01-10
3,990283065,2019-01-10,2020-01-09,NaT,Инфоном,None,ОМС предоплата,Вохгельд Михаил Рудольфович,1963-03-15
4,1990217763,2019-03-15,2020-03-14,NaT,"АО ""Группа Ренессанс Страхование""",Телемедицина_Ренес,Телемедицина,Ядгаров Артур Игоревич,1991-01-07
...,...,...,...,...,...,...,...,...,...
861569,998212426,2020-04-01,2021-03-31,2020-03-31,"Общество с ограниченной ответственностью СК ""А...",Альянс Жизнь_детский_ТМ,Телемедицина,Шпаковская Анастасия Павловна,1997-09-19
861570,997989026,2020-04-01,2021-03-31,2020-03-31,"Общество с ограниченной ответственностью СК ""А...",Альянс Жизнь_детский_ТМ,Телемедицина,Матвеева Василина Алексеевна,1997-12-20
861571,997989031,2020-04-01,2021-03-31,2020-03-31,"Общество с ограниченной ответственностью СК ""А...",Альянс Жизнь_детский_ТМ,Телемедицина,Плаксицкий Антон Павлович,1992-08-31
861572,998027564,2020-04-01,2021-03-31,2020-03-31,"Общество с ограниченной ответственностью СК ""А...",Альянс Жизнь_детский_ТМ,Телемедицина,Васильев Алексей Витальевич,1996-04-04


In [12]:
df_discards_temp = df_discards.groupby(['JNAME', 'SHORTNAME', 'TNAME', 'FULLNAME', 'BDATE'])[['PCODE']].count().reset_index().rename(columns={'PCODE':'DISCARDS'})

In [13]:
df_discards_temp.groupby(['JNAME', 'SHORTNAME', 'TNAME'])[['DISCARDS']].sum().reset_index()

,JNAME,SHORTNAME,TNAME,DISCARDS
0,"""МПОТК ""Технокомплект""",Телемедицина,Телемедицина,25
1,"АО ""Агрохимсервис""",телемедицина,Телемедицина,1
2,"АО ""Группа Ренессанс Страхование""",АПП дети 0-1 Ренесс,СК 1 факт,24
3,"АО ""Группа Ренессанс Страхование""",АПП дети 1-7_ренес,СК 1 факт,14
4,"АО ""Группа Ренессанс Страхование""",АПП дети 11-18_Ренес,СК 1 факт,144
...,...,...,...,...
652,ФЛ,ЧА: Мужское здоровье (6000),ФЛ предоплата,1
653,ФЛ,ЧА:Гастро. Лайт (4800),ФЛ предоплата,1
654,ФЛ,ЧА:Гастро. Максимум (10000),ФЛ предоплата,3
655,ФЛ,ЧА:Здоровая щитовидная железа (4 000),ФЛ предоплата,2


In [14]:
discards = df_discards_temp.groupby(['JNAME', 'SHORTNAME', 'TNAME'])[['DISCARDS']].sum().reset_index()

# NEW PATIENTS

In [15]:
sql_joins = f"""
SELECT	clhistnum.PCODE, -- ПАТИЕНТ_ИД
		clhistnum.BDATE, -- НАЧАЛО ПРИКРЕПЛЕНИЯ
		clhistnum.FDATE, -- КОНЕЦ ПРИКРЕПЛЕНИЯ
		clhistnum.DATECANCEL, -- ДОСРОЧНОЕ ОТКРЕПЛЕНИЕ
		jpersons.JNAME, -- ПАРТНЕР
		jplists.shortname, -- ДОГОВОР
		JPAGRTYPES.TNAME, -- ТИП ДОГОВОРА
		clients.FULLNAME,
		clients.BDATE birthday
		--
FROM clhistnum
--LEFT JOIN TREAT ON treat.pcode = clhistnum.PCODE
--
left JOIN jplists ON clhistnum.LSTID = JPLISTS.LID
LEFT JOIN JPAGREEMENT ON JPAGREEMENT.AGRID = CLHISTNUM.AGRID
LEFT JOIN JPAGRTYPES ON JPAGRTYPES.AGRTYPE=JPAGREEMENT.AGRTYPE
LEFT JOIN JPERSONS on JPAGREEMENT.jid = jpersons.JID
--
LEFT JOIN clients ON clients.PCODE = clhistnum.PCODE
--
WHERE 
clhistnum.BDATE >= '{period_start}' AND clhistnum.BDATE <= '{period_end}'
AND clhistnum.FDATE > '{period_end}'
"""

In [16]:
conn = UniversalConnection('../../../../.credentials/infoclinic/PROM.cfg', DBType.FireBird)
df_joins = conn.query(sql_joins)
conn.close()

2020-06-19 13:44:23 connect to firebird database using config file "../../../../.credentials/infoclinic/PROM.cfg"
2020-06-19 13:44:24 connected to database:
                    b:\ifk-fdb\maindb\099.fdb
2020-06-19 13:44:24 query firebird:
                    
				    SELECT	clhistnum.PCODE, -- ПАТИЕНТ_ИД
				    		clhistnum.BDATE, -- НАЧАЛО ПРИКРЕПЛЕНИЯ
				    		clhistnum.FDATE, -- КОНЕЦ ПРИКРЕПЛЕНИЯ
				    		clhistnum.DATECANCEL, -- ДОСРОЧНОЕ ОТКРЕПЛЕНИЕ
				    		jpersons.JNAME, -- ПАРТНЕР
				    		jplists.shortname, -- ДОГОВОР
				    		JPAGRTYPES.TNAME, -- ТИП ДОГОВОРА
				    		clients.FULLNAME,
				    		clients.BDATE birthday
				    		--
				    FROM clhistnum
				    --LEFT JOIN TREAT ON treat.pcode = clhistnum.PCODE
				    --
				    left JOIN jplists ON clhistnum.LSTID = JPLISTS.LID
				    LEFT JOIN JPAGREEMENT ON JPAGREEMENT.AGRID = CLHISTNUM.AGRID
				    LEFT JOIN JPAGRTYPES ON JPAGRTYPES.AGRTYPE=JPAGREEMENT.AGRTYPE
				    LEFT JOIN JPERSONS on JPAGREEMENT.jid = jpe

In [17]:
df_joins

,PCODE,BDATE,FDATE,DATECANCEL,JNAME,SHORTNAME,TNAME,FULLNAME,BIRTHDAY
0,990189873,2020-01-01,2020-12-31,2020-01-31,"Открытое акционерное общество ""АльфаСтрахование""",АПП+ПНД_Альфастрах,СК 1 факт,Ивашкова Лидия Владимировна,1963-08-26
1,990151286,2020-03-16,2021-03-16,NaT,Инфоном,None,ОМС предоплата,Селютина Елена Владимировна,1981-08-15
2,990324604,2020-01-01,2020-12-31,NaT,"Акционерное общество ""Страховое общество газов...",АПП+СТ_СОГАЗ,СК 1 факт,Алехин Владимир Петрович,1952-06-30
3,990680833,2020-02-05,2021-02-04,2020-03-19,"Открытое акционерное общество ""АльфаСтрахование""",АПП_Альфастрах,СК 1 факт,Абдуллин Игорь Валерьевич,1989-08-07
4,990680866,2020-02-05,2021-02-04,2020-03-15,"Открытое акционерное общество ""АльфаСтрахование""",АПП_Альфастрах,СК 1 факт,Ульмаскулова Мария Владимировна,1991-05-27
...,...,...,...,...,...,...,...,...,...
573053,991605210,2020-03-01,2020-08-31,NaT,"Общество с ограниченной ответственностью СК ""А...",Оптима АПП+ПНД Альянс,СК 1 факт,Ярошевский Виталий Валерьевич,1993-10-10
573054,990401407,2020-03-01,2020-08-31,NaT,"Общество с ограниченной ответственностью СК ""А...",Оптима АПП+ПНД Альянс,СК 1 факт,Ясеновский Андрей Александрович,1983-10-21
573055,993711976,2020-03-01,2020-08-31,NaT,"Общество с ограниченной ответственностью СК ""А...",Оптима АПП+ПНД Альянс,СК 1 факт,Ястремский Виталий Викторович,1973-05-07
573056,996833716,2020-03-01,2020-08-31,NaT,"Общество с ограниченной ответственностью СК ""А...",Оптима АПП+ПНД Альянс,СК 1 факт,Яцук Алина Николаевна,1997-03-16


In [18]:
df_joins_temp = df_joins.groupby(['JNAME', 'SHORTNAME', 'TNAME', 'FULLNAME', 'BDATE'])[['PCODE']].count().reset_index().rename(columns={'PCODE':'JOINS'})

In [19]:
df_joins_temp.groupby(['JNAME', 'SHORTNAME', 'TNAME'])[['JOINS']].sum().reset_index()

,JNAME,SHORTNAME,TNAME,JOINS
0,"АО ""Группа Ренессанс Страхование""",А АПП (10-50 чел),СК 1 предоплата,9
1,"АО ""Группа Ренессанс Страхование""",АПП дети 0-1 Ренесс,СК 1 факт,17
2,"АО ""Группа Ренессанс Страхование""",АПП дети 1-7_ренес,СК 1 факт,46
3,"АО ""Группа Ренессанс Страхование""",АПП дети 11-18_Ренес,СК 1 факт,32
4,"АО ""Группа Ренессанс Страхование""",АПП дети 7-11_Ренес,СК 1 факт,43
...,...,...,...,...
352,ФЛ,"ППП:Программа медобслуживания для взрослых ""Ба...",ФЛ предоплата,4
353,ФЛ,"ППП:Программа медобслуживания для взрослых ""Ба...",ФЛ предоплата,2
354,ФЛ,"ППП:Программа медобслуживания для взрослых ""Ст...",ФЛ предоплата,17
355,ФЛ,ППП:Программа медобслуживания для взрослых Ста...,ФЛ предоплата,1


In [20]:
joins = df_joins_temp.groupby(['JNAME', 'SHORTNAME', 'TNAME'])[['JOINS']].sum().reset_index()

# CURRENT PATIENTS

In [73]:
sql_current = f"""
SELECT	clhistnum.PCODE, -- ПАТИЕНТ_ИД
		clhistnum.BDATE, -- НАЧАЛО ПРИКРЕПЛЕНИЯ
		clhistnum.FDATE, -- КОНЕЦ ПРИКРЕПЛЕНИЯ
		clhistnum.DATECANCEL, -- ДОСРОЧНОЕ ОТКРЕПЛЕНИЕ
		jpersons.JNAME, -- ПАРТНЕР
		jplists.shortname, -- ДОГОВОР
		JPAGRTYPES.TNAME, -- ТИП ДОГОВОРА
		clients.FULLNAME,
		clients.BDATE birthday
		--
FROM clhistnum
--
left JOIN jplists ON clhistnum.LSTID = JPLISTS.LID
LEFT JOIN JPAGREEMENT ON JPAGREEMENT.AGRID = CLHISTNUM.AGRID
LEFT JOIN JPAGRTYPES ON JPAGRTYPES.AGRTYPE=JPAGREEMENT.AGRTYPE
LEFT JOIN JPERSONS on JPAGREEMENT.jid = jpersons.JID
--
LEFT JOIN clients ON clients.PCODE = clhistnum.PCODE
--
WHERE (
clhistnum.BDATE < '{period_start}' --period end
AND clhistnum.FDATE > '{period_end}' --period start
AND (clhistnum.DATECANCEL IS NULL OR (clhistnum.FDATE <= clhistnum.DATECANCEL AND clhistnum.FDATE > '{period_end}')))


"""

In [74]:
conn = UniversalConnection('../../../../.credentials/infoclinic/PROM.cfg', DBType.FireBird)
df_current = conn.query(sql_current)
conn.close()

2020-06-19 14:27:05 connect to firebird database using config file "../../../../.credentials/infoclinic/PROM.cfg"
2020-06-19 14:27:06 connected to database:
                    b:\ifk-fdb\maindb\099.fdb
2020-06-19 14:27:06 query firebird:
                    
				    SELECT	clhistnum.PCODE, -- ПАТИЕНТ_ИД
				    		clhistnum.BDATE, -- НАЧАЛО ПРИКРЕПЛЕНИЯ
				    		clhistnum.FDATE, -- КОНЕЦ ПРИКРЕПЛЕНИЯ
				    		clhistnum.DATECANCEL, -- ДОСРОЧНОЕ ОТКРЕПЛЕНИЕ
				    		jpersons.JNAME, -- ПАРТНЕР
				    		jplists.shortname, -- ДОГОВОР
				    		JPAGRTYPES.TNAME, -- ТИП ДОГОВОРА
				    		clients.FULLNAME,
				    		clients.BDATE birthday
				    		--
				    FROM clhistnum
				    --
				    left JOIN jplists ON clhistnum.LSTID = JPLISTS.LID
				    LEFT JOIN JPAGREEMENT ON JPAGREEMENT.AGRID = CLHISTNUM.AGRID
				    LEFT JOIN JPAGRTYPES ON JPAGRTYPES.AGRTYPE=JPAGREEMENT.AGRTYPE
				    LEFT JOIN JPERSONS on JPAGREEMENT.jid = jpersons.JID
				    --
				    LEFT JOIN clients ON clients.P

In [75]:
df_current

,PCODE,BDATE,FDATE,DATECANCEL,JNAME,SHORTNAME,TNAME,FULLNAME,BIRTHDAY
0,90000105,2015-12-19,2100-12-19,NaT,ФЛ,"Скидка:Скидка для ветеранов, инвалидов по обсл...",None,Грибкова Надежда Павловна,1957-03-07
1,995297417,2019-04-18,2020-04-14,NaT,ФЛ,ЧА:Медосмотр в сад (10000),ФЛ предоплата,Егоркин Илья Сергеевич,2015-12-03
2,990130901,2019-05-07,2021-05-06,NaT,Инфоном,None,ОМС предоплата,Воротягина Наталья Сергеевна,1985-03-21
3,990146740,2019-04-03,2020-04-02,NaT,ФЛ,"ППП:Программа медобслуживания для взрослых ""Ба...",ФЛ предоплата,Жикунова Елена Васильевна,1966-07-26
4,90001461,2018-09-15,2020-08-14,2020-08-14,ФЛ,ППП:Доктор рядом - ведение беременности. I-III...,ФЛ предоплата,Одинцова Татьяна Анатольевна,1985-02-18
...,...,...,...,...,...,...,...,...,...
5213305,993055221,2019-07-01,2020-06-30,NaT,"Общество с ограниченной ответственностью ""Стра...",Телемедицина_Согласие_МКДР,Телемедицина,Дымов Вадим Георгиевич,1971-08-27
5213306,990458239,2019-07-31,2020-07-15,NaT,"Общество с ограниченной ответственностью ""Росг...",РГС Телемедицина ФЛ + псих_Навигация по ОМС,Телемедицина,Белоног Елена Игоревна,1989-07-16
5213307,998676466,2019-09-01,2020-08-31,NaT,Техническая программа,ТМ_Промокод_все врачи,Телемедицина,Акопян Ольга Николаевна,1993-02-15
5213308,999131375,2019-10-01,2020-09-30,NaT,Техническая программа,ТМ_Промокод_все врачи,Телемедицина,Анищенко Татьяна Алексеевна,1990-08-13


In [76]:
df_current_temp = df_current.groupby(['JNAME', 'SHORTNAME', 'TNAME', 'FULLNAME', 'BDATE'])[['PCODE']].count().reset_index().rename(columns={'PCODE':'JOINS'})

In [77]:
current = df_current.groupby(['JNAME', 'SHORTNAME', 'TNAME'])[['PCODE']].count().reset_index().rename(columns={'PCODE':'CURRENT_TEMP'})

In [78]:
current.head()

,JNAME,SHORTNAME,TNAME,CURRENT_TEMP
0,"АО ""Башмаковский хлеб""",Диет_Основы питания. Базовый_МКДР,Телемедицина,6
1,"АО ""Башмаковский хлеб""",ТЛЧК_Стандарт. Ежегодное обследование для мужч...,Телемедицина,7
2,ИП Аносенкова Оксана Дмитриевна,Телемедицина_МКДР,Телемедицина,1
3,ИП Волчкова Алла Михайловна,Телемедицина,Телемедицина,3
4,ИП Глава КФХ Толоконников Сергей Анатольевич,Телемедицина,Телемедицина,1


In [79]:
# joins[joins['JNAME'] == 'АО "Группа Ренессанс Страхование"']

In [80]:
# current[current['JNAME'] == 'АО "Группа Ренессанс Страхование"']

In [132]:
merge_temp = current.merge(joins, how='left')
merge_temp.JOINS = merge_temp.JOINS.fillna(0).astype(int)

In [133]:
merged = merge_temp.merge(discards, how='left')
merged.DISCARDS = merged.DISCARDS.fillna(0).astype(int)

In [134]:
merged['ON_START'] = merged['CURRENT_TEMP'] + merged['DISCARDS']
merged['ON_END'] = merged['CURRENT_TEMP'] + merged['JOINS']
#merged['ON_END'] = merged['ON_START'] - merged['DISCARDS'] + merged['JOINS']

In [135]:
merged.tail(10)

,JNAME,SHORTNAME,TNAME,CURRENT_TEMP,JOINS,DISCARDS,ON_START,ON_END
1691,ФЛ,ППП:Комплексная программа «Педиатр на дому+» о...,ФЛ предоплата,1,0,0,1,1
1692,ФЛ,ППП:Личный врач+ (120000),ФЛ предоплата,1,0,0,1,1
1693,ФЛ,"ППП:Программа медобслуживания для взрослых ""Ба...",ФЛ предоплата,15,4,5,20,19
1694,ФЛ,"ППП:Программа медобслуживания для взрослых ""Ба...",ФЛ предоплата,5,0,1,6,5
1695,ФЛ,"ППП:Программа медобслуживания для взрослых ""Ст...",ФЛ предоплата,27,17,33,60,44
1696,ФЛ,ППП:Программа медобслуживания для взрослых Ста...,ФЛ предоплата,5,1,3,8,6
1697,ФЛ,ППП:Программа медобслуживания для взрослых Ста...,ФЛ предоплата,2,0,0,2,2
1698,ФЛ,ЧА:Акция у Уролога (3000),ФЛ предоплата,1,0,0,1,1
1699,ФЛ,ЧА:Выгодная диагностика желудка (2500),ФЛ предоплата,1,0,0,1,1
1700,ФЛ,ЧА:Медосмотр в сад (10000),ФЛ предоплата,2,0,0,2,2


In [136]:
# merged.to_excel('merged.xlsx', index=False)

In [137]:
merged[merged['JNAME'] == 'Акционерное общество "СГ Уралсиб"']

,JNAME,SHORTNAME,TNAME,CURRENT_TEMP,JOINS,DISCARDS,ON_START,ON_END
142,"Акционерное общество ""СГ Уралсиб""",АПП+СТ Уралсиб,СК 1 факт,7,0,0,7,7
143,"Акционерное общество ""СГ Уралсиб""",АПП_Уралсиб,СК 1 факт,12,248,8,20,260
144,"Акционерное общество ""СГ Уралсиб""",Бизнес.Здоровый коллектив_ТМ ТЛЧК_МКДР,Телемедицина,62,92,0,62,154
145,"Акционерное общество ""СГ Уралсиб""",Онкоконтроль.Здоровый коллектив_ТМ_МКДР,Телемедицина,1,0,0,1,1
146,"Акционерное общество ""СГ Уралсиб""",Стандарт.Здоровый коллектив_ТМ ТЛЧК_МКДР,Телемедицина,863,1099,0,863,1962
147,"Акционерное общество ""СГ Уралсиб""",ТМ_ТелеДоктор УралСиб,Телемедицина,30,0,1323,1353,30
148,"Акционерное общество ""СГ Уралсиб""",ТМ_ТелеДоктор УралСиб_техническая,Телемедицина,1479,0,5075,6554,1479
149,"Акционерное общество ""СГ Уралсиб""",ТМ_ТелеДоктор_УралСиб,Телемедицина,6,0,1,7,6
150,"Акционерное общество ""СГ Уралсиб""",ТМ_ТелеДоктор_Уралсиб,Телемедицина,13996,2904,93,14089,16900
151,"Акционерное общество ""СГ Уралсиб""",ТМ_ТелеДоктор_Уралсиб_2,Телемедицина,17816,1290,13,17829,19106


# CONSULTATIONS

In [138]:
sql_cons = f"""
SELECT 	TREAT.TREATCODE,
		TREAT.TREATDATE,
		--clhistnum.MAINJID,
		--clhistnum.PARENTHISTID,
		jpersons.JNAME,
		clhistnum.PCODE, -- ПАТИЕНТ_ИД
		--clhistnum.BDATE, -- НАЧАЛО ПРИКРЕПЛЕНИЯ
		--clhistnum.FDATE, -- КОНЕЦ ПРИКРЕПЛЕНИЯ
		--clhistnum.DATECANCEL, -- ДОСРОЧНОЕ ОТКРЕПЛЕНИЕ
		jplists.shortname, -- ДОГОВОР
		jpagreement.agnum, -- № ДОГОВОРА
		JPAGRTYPES.TNAME--, -- ТИП ДОГОВОРА
		--clhistnum.AMOUNTRUB -- ДЕНЬГИ
FROM TREAT
INNER JOIN clhistnum ON TREAT.HISTID = CLHISTNUM.HISTID 
					 AND TREAT.PCODE = CLHISTNUM.PCODE 
--inner JOIN clients ON TREAT.PCODE = clients.PCODE 
--JOIN JPERSONS ON TREAT.CLINICID = jpersons.jid
INNER JOIN JPERSONS ON CLHISTNUM.JID = jpersons.JID 
--
--
--left JOIN CLHISTNUM ON JPAGREEMENT.AGRID = CLHISTNUM.AGRID
inner JOIN jplists ON clhistnum.LSTID = JPLISTS.LID
--inner JOIN JPAGREEMENT ON JPAGREEMENT.jid = jpersons.JID
--
INNER JOIN jpagreement ON CLHISTNUM.AGRID = jpagreement.AGRID	
--
inner JOIN JPAGRTYPES ON JPAGRTYPES.AGRTYPE = JPAGREEMENT.AGRTYPE
--
--WHERE TREAT.TREATCODE = 60595284
WHERE (TREAT.TREATDATE >= '{period_start}'
AND TREAT.TREATDATE <= '{period_end}') --OR (TREAT.TREATDATE IS NULL AND clhistnum.PCODE IS NOT NULL))
--
AND (clhistnum.FDATE > '{period_start}' AND (clhistnum.DATECANCEL IS NULL OR TREAT.TREATDATE <= clhistnum.DATECANCEL))



"""

In [39]:
conn = UniversalConnection('../../../../.credentials/infoclinic/PROM.cfg', DBType.FireBird)

df_cons = conn.query(sql_cons)

conn.close()

2020-06-19 14:20:10 connect to firebird database using config file "../../../../.credentials/infoclinic/PROM.cfg"
2020-06-19 14:20:10 connected to database:
                    b:\ifk-fdb\maindb\099.fdb
2020-06-19 14:20:10 query firebird:
                    
				    SELECT 	TREAT.TREATCODE,
				    		TREAT.TREATDATE,
				    		--clhistnum.MAINJID,
				    		--clhistnum.PARENTHISTID,
				    		jpersons.JNAME,
				    		clhistnum.PCODE, -- ПАТИЕНТ_ИД
				    		--clhistnum.BDATE, -- НАЧАЛО ПРИКРЕПЛЕНИЯ
				    		--clhistnum.FDATE, -- КОНЕЦ ПРИКРЕПЛЕНИЯ
				    		--clhistnum.DATECANCEL, -- ДОСРОЧНОЕ ОТКРЕПЛЕНИЕ
				    		jplists.shortname, -- ДОГОВОР
				    		jpagreement.agnum, -- № ДОГОВОРА
				    		JPAGRTYPES.TNAME--, -- ТИП ДОГОВОРА
				    		--clhistnum.AMOUNTRUB -- ДЕНЬГИ
				    FROM TREAT
				    INNER JOIN clhistnum ON TREAT.HISTID = CLHISTNUM.HISTID 
				    					 AND TREAT.PCODE = CLHISTNUM.PCODE 
				    --inner JOIN clients ON TREAT.PCODE = clients.PCODE 
				    --JOIN JPER

In [40]:
# conn.close()

In [139]:
df_cons

,TREATCODE,TREATDATE,JNAME,PCODE,SHORTNAME,AGNUM,TNAME
0,980241295,2020-03-24,"АО ""Группа Ренессанс Страхование""",990412595,Телемедицина_Ренес,Доп.согл.№15,Телемедицина
1,980241326,2020-03-24,"АО ""Группа Ренессанс Страхование""",997318843,Телемедицина_Ренессанс МКДР,МКДР ДС№1 от 14.06.2019 к Дог. №30-19 от 01.06...,Телемедицина
2,980241332,2020-03-24,"ООО ""ДокДок ТЗ""",995346051,ТМ_ДМС_Сбербанк,28-Т3/19 от 09.01.2019,Телемедицина
3,980173534,2020-01-01,"АО ""Группа Ренессанс Страхование""",994851047,Телемедицина_Ренес,Доп.согл.№15,Телемедицина
4,980173537,2020-01-01,"АО ""Группа Ренессанс Страхование""",990461375,Телемедицина_Ренессанс МКДР,МКДР ДС№1 от 14.06.2019 к Дог. №30-19 от 01.06...,Телемедицина
...,...,...,...,...,...,...,...
143030,980251082,2020-03-04,Техническая программа,995903729,ТМ_Промокод_все врачи,1_технический,Телемедицина
143031,980251086,2020-03-04,"ООО ""ДокДок ТЗ""",995371122,ТМ_ДМС_Сбербанк,28-Т3/19 от 09.01.2019,Телемедицина
143032,980251089,2020-03-04,"ООО ""ДокДок ТЗ""",996825934,ТМ_ДМС_Сбербанк,28-Т3/19 от 09.01.2019,Телемедицина
143033,980251095,2020-03-04,"Общество с ограниченной ответственностью СК ""А...",996711669,Альянс Жизнь_взрослый_ТМ,МК ДР_Дог.№Л-135219/NV-032 ОТ 29.11.2019_(факт...,Телемедицина


In [140]:
df_cons.groupby(['JNAME', 'SHORTNAME', 'TNAME'])[['TREATCODE']].count().reset_index().rename(columns={'TREATCODE':'CONS_AMOUNT'})

,JNAME,SHORTNAME,TNAME,CONS_AMOUNT
0,ИП Гофман Наталья Викторовна,Диет_Основы питания. Стандарт_МКДР,Телемедицина,1
1,"""ООО ""ГК АВТОКЛАСС""",VIP.VIP-обследование для женщин_МК ДР,Телемедицина,1
2,"""ООО ""ЭкспертЛизинг""",ТЛЧК_Базовый. Профилактическое обследование до...,Телемедицина,7
3,Ready4med (ИП Жильцова),Индивидуальная ТМ.Расширенный пакет_МКДР,Телемедицина,1
4,Ready4med (ИП Жильцова),Семейная ТМ.Базовый пакет_МКДР,Телемедицина,3
...,...,...,...,...
405,ФЛ,ЧА: Кардио. Лайт 2.0 (5000),ФЛ предоплата,3
406,ФЛ,ЧА: Мужское здоровье (6000),ФЛ предоплата,1
407,ФЛ,ЧА:Гастро. Лайт (4800),ФЛ предоплата,5
408,ФЛ,ЧА:Здоровая щитовидная железа (4 000),ФЛ предоплата,10


In [141]:
cons = df_cons.groupby(['JNAME', 'SHORTNAME', 'TNAME'])[['TREATCODE']].count().reset_index().rename(columns={'TREATCODE':'CONS_AMOUNT'})

In [142]:
merged = merged.merge(cons, how='left')
merged.CONS_AMOUNT = merged.CONS_AMOUNT.fillna(0).astype(int)

In [143]:
merged.CONS_AMOUNT.sum()

140443

In [144]:
merged

,JNAME,SHORTNAME,TNAME,CURRENT_TEMP,JOINS,DISCARDS,ON_START,ON_END,CONS_AMOUNT
0,"АО ""Башмаковский хлеб""",Диет_Основы питания. Базовый_МКДР,Телемедицина,6,0,0,6,6,0
1,"АО ""Башмаковский хлеб""",ТЛЧК_Стандарт. Ежегодное обследование для мужч...,Телемедицина,7,0,0,7,7,0
2,ИП Аносенкова Оксана Дмитриевна,Телемедицина_МКДР,Телемедицина,1,0,0,1,1,0
3,ИП Волчкова Алла Михайловна,Телемедицина,Телемедицина,3,0,0,3,3,0
4,ИП Глава КФХ Толоконников Сергей Анатольевич,Телемедицина,Телемедицина,1,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...
1696,ФЛ,ППП:Программа медобслуживания для взрослых Ста...,ФЛ предоплата,5,1,3,8,6,75
1697,ФЛ,ППП:Программа медобслуживания для взрослых Ста...,ФЛ предоплата,2,0,0,2,2,4
1698,ФЛ,ЧА:Акция у Уролога (3000),ФЛ предоплата,1,0,0,1,1,0
1699,ФЛ,ЧА:Выгодная диагностика желудка (2500),ФЛ предоплата,1,0,0,1,1,0


In [145]:
len(df_cons.PCODE.unique())

25364

In [146]:
df_cons.groupby(['JNAME', 'SHORTNAME', 'TNAME'])[['PCODE']].nunique().reset_index().rename(columns={'PCODE':'UNIQUE_USERS'})

,JNAME,SHORTNAME,TNAME,UNIQUE_USERS
0,ИП Гофман Наталья Викторовна,Диет_Основы питания. Стандарт_МКДР,Телемедицина,1
1,"""ООО ""ГК АВТОКЛАСС""",VIP.VIP-обследование для женщин_МК ДР,Телемедицина,1
2,"""ООО ""ЭкспертЛизинг""",ТЛЧК_Базовый. Профилактическое обследование до...,Телемедицина,1
3,Ready4med (ИП Жильцова),Индивидуальная ТМ.Расширенный пакет_МКДР,Телемедицина,1
4,Ready4med (ИП Жильцова),Семейная ТМ.Базовый пакет_МКДР,Телемедицина,1
...,...,...,...,...
405,ФЛ,ЧА: Кардио. Лайт 2.0 (5000),ФЛ предоплата,1
406,ФЛ,ЧА: Мужское здоровье (6000),ФЛ предоплата,1
407,ФЛ,ЧА:Гастро. Лайт (4800),ФЛ предоплата,1
408,ФЛ,ЧА:Здоровая щитовидная железа (4 000),ФЛ предоплата,2


In [147]:
unique_users = df_cons.groupby(['JNAME', 'SHORTNAME', 'TNAME'])[['PCODE']].nunique().reset_index().rename(columns={'PCODE':'UNIQUE_USERS'})

In [148]:
unique_users.UNIQUE_USERS.sum()

25766

In [149]:
merged = merged.merge(unique_users, how='left')
merged.UNIQUE_USERS = merged.UNIQUE_USERS.fillna(0).astype(int)

In [151]:
merged

,JNAME,SHORTNAME,TNAME,CURRENT_TEMP,JOINS,DISCARDS,ON_START,ON_END,CONS_AMOUNT,UNIQUE_USERS
0,"АО ""Башмаковский хлеб""",Диет_Основы питания. Базовый_МКДР,Телемедицина,6,0,0,6,6,0,0
1,"АО ""Башмаковский хлеб""",ТЛЧК_Стандарт. Ежегодное обследование для мужч...,Телемедицина,7,0,0,7,7,0,0
2,ИП Аносенкова Оксана Дмитриевна,Телемедицина_МКДР,Телемедицина,1,0,0,1,1,0,0
3,ИП Волчкова Алла Михайловна,Телемедицина,Телемедицина,3,0,0,3,3,0,0
4,ИП Глава КФХ Толоконников Сергей Анатольевич,Телемедицина,Телемедицина,1,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...
1696,ФЛ,ППП:Программа медобслуживания для взрослых Ста...,ФЛ предоплата,5,1,3,8,6,75,5
1697,ФЛ,ППП:Программа медобслуживания для взрослых Ста...,ФЛ предоплата,2,0,0,2,2,4,1
1698,ФЛ,ЧА:Акция у Уролога (3000),ФЛ предоплата,1,0,0,1,1,0,0
1699,ФЛ,ЧА:Выгодная диагностика желудка (2500),ФЛ предоплата,1,0,0,1,1,0,0


In [152]:
# merged.to_excel('UTILIZE.xlsx')

In [153]:
merged['CONS_ON_USER'] = merged['CONS_AMOUNT'] / merged['UNIQUE_USERS']
merged.CONS_ON_USER = merged.CONS_ON_USER.fillna(0).astype(float)

In [154]:
merged['UTILIZE_old'] = merged['CONS_AMOUNT'] / ((merged['ON_END'] + merged['ON_START']) / 2)

In [155]:
merged

,JNAME,SHORTNAME,TNAME,CURRENT_TEMP,JOINS,DISCARDS,ON_START,ON_END,CONS_AMOUNT,UNIQUE_USERS,CONS_ON_USER,UTILIZE_old
0,"АО ""Башмаковский хлеб""",Диет_Основы питания. Базовый_МКДР,Телемедицина,6,0,0,6,6,0,0,0.0,0.000000
1,"АО ""Башмаковский хлеб""",ТЛЧК_Стандарт. Ежегодное обследование для мужч...,Телемедицина,7,0,0,7,7,0,0,0.0,0.000000
2,ИП Аносенкова Оксана Дмитриевна,Телемедицина_МКДР,Телемедицина,1,0,0,1,1,0,0,0.0,0.000000
3,ИП Волчкова Алла Михайловна,Телемедицина,Телемедицина,3,0,0,3,3,0,0,0.0,0.000000
4,ИП Глава КФХ Толоконников Сергей Анатольевич,Телемедицина,Телемедицина,1,0,0,1,1,0,0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
1696,ФЛ,ППП:Программа медобслуживания для взрослых Ста...,ФЛ предоплата,5,1,3,8,6,75,5,15.0,10.714286
1697,ФЛ,ППП:Программа медобслуживания для взрослых Ста...,ФЛ предоплата,2,0,0,2,2,4,1,4.0,2.000000
1698,ФЛ,ЧА:Акция у Уролога (3000),ФЛ предоплата,1,0,0,1,1,0,0,0.0,0.000000
1699,ФЛ,ЧА:Выгодная диагностика желудка (2500),ФЛ предоплата,1,0,0,1,1,0,0,0.0,0.000000


In [156]:
merged['UTILIZE_CONS'] = merged['CONS_AMOUNT'] / (merged['CURRENT_TEMP'] + merged['JOINS'] + merged['DISCARDS'])

In [157]:
merged['UTILIZE_UNIQUE_USERS'] = merged['UNIQUE_USERS'] / (merged['CURRENT_TEMP'] + merged['JOINS'] + merged['DISCARDS'])

In [158]:
merged

,JNAME,SHORTNAME,TNAME,CURRENT_TEMP,JOINS,DISCARDS,ON_START,ON_END,CONS_AMOUNT,UNIQUE_USERS,CONS_ON_USER,UTILIZE_old,UTILIZE_CONS,UTILIZE_UNIQUE_USERS
0,"АО ""Башмаковский хлеб""",Диет_Основы питания. Базовый_МКДР,Телемедицина,6,0,0,6,6,0,0,0.0,0.000000,0.000000,0.000000
1,"АО ""Башмаковский хлеб""",ТЛЧК_Стандарт. Ежегодное обследование для мужч...,Телемедицина,7,0,0,7,7,0,0,0.0,0.000000,0.000000,0.000000
2,ИП Аносенкова Оксана Дмитриевна,Телемедицина_МКДР,Телемедицина,1,0,0,1,1,0,0,0.0,0.000000,0.000000,0.000000
3,ИП Волчкова Алла Михайловна,Телемедицина,Телемедицина,3,0,0,3,3,0,0,0.0,0.000000,0.000000,0.000000
4,ИП Глава КФХ Толоконников Сергей Анатольевич,Телемедицина,Телемедицина,1,0,0,1,1,0,0,0.0,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1696,ФЛ,ППП:Программа медобслуживания для взрослых Ста...,ФЛ предоплата,5,1,3,8,6,75,5,15.0,10.714286,8.333333,0.555556
1697,ФЛ,ППП:Программа медобслуживания для взрослых Ста...,ФЛ предоплата,2,0,0,2,2,4,1,4.0,2.000000,2.000000,0.500000
1698,ФЛ,ЧА:Акция у Уролога (3000),ФЛ предоплата,1,0,0,1,1,0,0,0.0,0.000000,0.000000,0.000000
1699,ФЛ,ЧА:Выгодная диагностика желудка (2500),ФЛ предоплата,1,0,0,1,1,0,0,0.0,0.000000,0.000000,0.000000


In [125]:
# merged = merged.drop(['UTILIZE'], axis=1)

In [124]:
# merged[merged.UTILIZE > 0]

In [133]:
# merged.to_excel('UTILIZE.xlsx', index=False)

# SAVE

In [182]:
merged = merged.drop(['CURRENT_TEMP'], axis=1)

In [183]:
report_date: str = datetime.datetime.now().strftime(DATETIME_FORMAT).replace(':', '-')

In [184]:
#report_date

In [185]:
global writer
writer = None

In [186]:
writer = auto_columns_width('UTILIZE_' + report_date + '.xlsx', merged, 'Data', writer)

0 JNAME 51.0
1 SHORTNAME 51.0
2 TNAME 25.5
3 JOINS 11.9
4 DISCARDS 13.6
5 ON_START 13.6
6 ON_END 11.9
7 CONS_AMOUNT 18.7
8 UNIQUE_USERS 20.4
9 CONS_ON_USER 30.599999999999998
10 UTILIZE_old 37.4
11 UTILIZE_CONS 37.4
12 UTILIZE_UNIQUE_USERS 37.4


In [187]:
writer.save()

In [65]:
# type(merged.index)

# CORR_UNIQ_USERS

In [170]:
dis_pcodes = list(df_discards.PCODE.unique())

In [171]:
joins_pcodes = list(df_joins.PCODE.unique())

In [173]:
pcodes = list(set(dis_pcodes) & set(joins_pcodes))

In [174]:
len(pcodes)

134997

In [179]:
df_discards.head()

,PCODE,BDATE,FDATE,DATECANCEL,JNAME,SHORTNAME,TNAME,FULLNAME,BIRTHDAY,pseudo_code
0,990159928,2019-02-12,2020-02-11,NaT,ФЛ,"ППП:Программа медобслуживания для взрослых ""Ст...",ФЛ предоплата,Балахнина Елена Анатольевна,1959-04-07,Балахнина Елена Анатольевна1959-04-07 00:00:00
1,995133290,2019-02-01,2020-01-31,NaT,Закрытое акционерное общество «МетЛайф»,АПП Метлайф,СК 1 факт,Двинских Александр Эдуардович,1993-09-05,Двинских Александр Эдуардович1993-09-05 00:00:00
2,990129261,2019-01-15,2020-01-16,NaT,ФЛ,"ППП:Программа медобслуживания для взрослых ""Ст...",ФЛ предоплата,Замятина Ольга Игоревна,1987-01-10,Замятина Ольга Игоревна1987-01-10 00:00:00
3,990283065,2019-01-10,2020-01-09,NaT,Инфоном,None,ОМС предоплата,Вохгельд Михаил Рудольфович,1963-03-15,Вохгельд Михаил Рудольфович1963-03-15 00:00:00
4,1990217763,2019-03-15,2020-03-14,NaT,"АО ""Группа Ренессанс Страхование""",Телемедицина_Ренес,Телемедицина,Ядгаров Артур Игоревич,1991-01-07,Ядгаров Артур Игоревич1991-01-07 00:00:00


In [178]:
df_discards['pseudo_code'] = df_discards['FULLNAME'] + df_discards['BIRTHDAY'].apply(str)

In [180]:
df_joins['pseudo_code'] = df_joins['FULLNAME'] + df_joins['BIRTHDAY'].apply(str)

In [181]:
dis_ps_codes = list(df_discards.pseudo_code.unique())
joins_ps_codes = list(df_joins.pseudo_code.unique())

pscodes = list(set(dis_ps_codes) & set(joins_ps_codes))
len(pscodes)

135043